In [84]:
# Módulos
import ee
import folium
import geehydro

In [85]:
# Inicio de API
ee.Initialize()

#### Función generateGreedForGeometry

In [86]:
#funcion para la generacion de grillas
def generateGreedForGeometry(geom,dx,dy):
    """
    Función para generar grillas partiendo de una zona geográfica.
    """

    bounds = ee.Geometry(geom).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    xx = ee.List.sequence(xmin, xmax, dx)
    yy = ee.List.sequence(ymin, ymax, dy)

    #************************************
    def contYYa(y,x,dx,dy):

        x1 = ee.Number(x)
        x2 = ee.Number(x).add(ee.Number(dx))
        y1 = ee.Number(y)
        y2 = ee.Number(y).add(ee.Number(dy))

        coords = ee.List([x1, y1, x2, y2])
        rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)

        return ee.Feature(rect)

    def contYY(x):
        return yy.map(lambda yi: contYYa(y=yi,x=x,dx=dx,dy=dy) )
    #************************************

    cells = xx.map(contYY).flatten() 

    return ee.FeatureCollection(cells).filterBounds(geom)#.map(lambda c: ee.Feature(c).intersection(geom) )

#### Ejemplo

In [87]:
#----------------------------------------------
# Ejemplo
#----------------------------------------------

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Canaima National Park'#'Guatopo'

# date
date_ini = '2019-12-01'
date_end = '2019-12-31'


# poligono
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))


# geometria
state = polygon.geometry()
dx = 0.09
dy = 0.09

# creando grilla
greed = generateGreedForGeometry(state,dx,dy)
centroide = state.centroid().coordinates().getInfo()[::-1]

#### Creando centroides

In [88]:
def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)


centroids = greed.map(getCentroid)

print('área del parque Km2: ', 0.1*polygon.first().geometry().area().divide(100*100).getInfo() ) 
print('área de cuadricula Km2: ', 0.1*greed.first().geometry().area().divide(100*100).getInfo() )
print('número de centroides: ', centroids.size().getInfo())

área del parque Km2:  289542.4562322496
área de cuadricula Km2:  997.0842463804966
número de centroides:  346


#### Información Indices de vegetacion

In [89]:
def getQABits(image, start, end, newName):
    """
    Funcion para establecer la calidad de las bandas
    """

    import math

    pattern = 0
    for i in range(start,end+1):
      pattern += math.pow(2, i)
    #Return a single band image of the extracted QA bits, giving the band
    #a new name.
    return image.select([0], [newName])\
                  .bitwiseAnd(pattern)\
                  .rightShift(start)

#### MOD13Q1

In [90]:
# MODIS MOD13Q1
modis = ee.ImageCollection('MODIS/006/MOD13Q1')\
             .filterDate(ee.DateRange(date_ini,date_end))\
             .map(lambda x: x.clip(polygon))


# Filtros de calidad -----------------------------------------
# def modis_bit_0(image):

#   qscore = getQABits(image.select('SummaryQA'), 0, 1, 'quality')\
#               .expression("b(0) == 2 || b(0) == 3" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_1(image):

#   qscore = getQABits(image.select('SummaryQA'), 2, 5, 'usefulness')\
#               .expression("b(0) == 2 || b(0) == 4 || b(0) == 8 || b(0) == 9 || b(0) == 10 || b(0) == 12 || b(0) == 13 || b(0) == 14 || b(0) == 15" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_2(image):

#   qscore = getQABits(image.select('SummaryQA'), 13, 14, 'Land/water')\
#               .expression("b(0) == 0 || b(0) == 1 || b(0) == 2 || b(0) == 3 || b(0) == 4 || b(0) == 5 || b(0) == 6 || b(0) == 7" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_3(image):

#   qscore = getQABits(image.select('SummaryQA'), 14, 14, 'snow/ice')\
#               .expression("b(0) == 1" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_4(image):

#   qscore = getQABits(image.select('DetailedQA'), 15, 15, 'shadow')\
#               .expression("b(0) == 1" )

#   return image.updateMask(qscore.Not())

# Filtros de calidad -----------------------------------------


def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])



In [91]:
# select EVI and NDVI
modis_qa = modis#.map(modis_bit_0)
                # .map(modis_bit_1)\
                # .map(modis_bit_2)\
                # .map(modis_bit_3)\
                # .map(modis_bit_4)
                
evi = modis_qa.select('EVI').map(scale_factor).mean()
ndvi = modis_qa.select('NDVI').map(scale_factor).mean()

#### SRTM Digital Elevation Data Version 4

In [92]:
# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation).clip(polygon)


MCD64A1.006 MODIS Burned Area Monthly Global 500m

In [93]:
fire = ee.ImageCollection('MODIS/006/MCD64A1')\
            .filter(ee.Filter.date(date_ini,date_end))\
            .map(lambda x:x.clip(polygon))



# Filtros de calidad -----------------------------------------
def bit_0(image):

  qscore = getQABits(image.select('QA'), 0, 0, 'Land_water')\
              .expression("b(0) == 0")

  return image.updateMask(qscore.Not())

#Select the QA band.
def bit_1(image):

  qscore = getQABits(image.select('QA'), 1, 1, 'Valid_data_flag')\
              .expression("b(0) == 0")

  return image.updateMask(qscore.Not())

# Filtros de calidad -----------------------------------------

burnedArea = fire.map(bit_0)\
                 .map(bit_1)\
                 .select('BurnDate')

#### Mapa

In [94]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')

Map.addLayer(slope, {min: 0, max: 60}, 'Slope')
Map.addLayer(ndvi
            ,name='NDVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['953302', '62D759','499443']}) 
Map.addLayer(evi
            ,name='EVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['953302', '62D759','499443']}) 
Map.addLayer(burnedArea, {'min': 30.0,
                        'max': 341.0,
                        'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
                        }, 'BurnedArea')
                        
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Centroids')

Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

#### Series temporales

##### Índices de vegetación

In [95]:
initDate = '2015-01-01' # Fecha de inicio
endDate = '2020-05-01' # Fecha final

In [96]:
#Filtros de calidad -----------------------------------------
def modis_bit_0(image):

  qscore = getQABits(image.select('SummaryQA'), 0, 1, 'quality')\
              .expression("b(0) == 1 || b(0) == 2 || b(0) == 3" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_1(image):

  qscore = getQABits(image.select('DetailedQA'), 2, 5, 'usefulness')\
              .expression("b(0) == 13 || b(0) == 14 || b(0) == 15" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_2(image):

  qscore = getQABits(image.select('DetailedQA'), 11, 13, 'Land/water')\
              .expression("b(0) == 0 || b(0) == 1 || b(0) == 2 || b(0) == 3 || b(0) == 4 || b(0) == 5 || b(0) == 6 || b(0) == 7" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_3(image):

  qscore = getQABits(image.select('DetailedQA'), 14, 14, 'snow/ice')\
              .expression("b(0) == 1" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_4(image):

  qscore = getQABits(image.select('DetailedQA'), 15, 15, 'shadow')\
              .expression("b(0) == 1" )

  return image.updateMask(qscore.Not())

#Filtros de calidad -----------------------------------------


def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])

In [97]:
MOD13Q1Collection = ee.ImageCollection('MODIS/006/MOD13Q1')\
                      .filterDate(ee.DateRange(initDate, endDate))\
                      .map(lambda x: x.clip(polygon))\
                      .map(modis_bit_0)\
                      .map(scale_factor)
                    #   .map(modis_bit_1)\
                    #   .map(modis_bit_2)\
                    #   .map(modis_bit_3)\
                    #   .map(modis_bit_4)

In [98]:
from ipygee import chart

# Media del EVI en polígono
EVI_serie = chart.Image.series(**{'imageCollection': MOD13Q1Collection,
                                  'region': polygon,
                                  'reducer': 'mean',
                                  'scale': 1000,
                                  'xProperty': 'system:time_start',
                                  'bands':'EVI'}).dataframe
# Media del NDVI en polígono
NDVI_serie = chart.Image.series(**{'imageCollection': MOD13Q1Collection,
                                    'region': polygon,
                                    'reducer': 'mean',
                                    'scale': 1000,
                                    'xProperty': 'system:time_start',
                                    'bands':'NDVI'}).dataframe

In [99]:
# agregando el tiempo
EVI_serie['date'] = EVI_serie.index 
NDVI_serie['date'] = NDVI_serie.index 

In [100]:
import plotly.graph_objects as go

# Create traces
fig_evi = go.Figure()
fig_evi.add_trace(go.Scatter(x=EVI_serie.date.tolist(), y=EVI_serie.EVI.tolist(),
                    mode='lines+markers',
                    name='EVI Time Serie',
                    line=dict(color='#78D759', width=4)
                    ))

# Edit the layout
fig_evi.update_layout(title='Serie de tiempo EVI',
                      xaxis_title='Mes',
                      yaxis_title='Índice de Vegetación')

fig_evi.show()

In [101]:
# Create traces
fig_ndvi = go.Figure()
fig_ndvi.add_trace(go.Scatter(
                    x=NDVI_serie.date.tolist(),
                    y=NDVI_serie.NDVI.tolist(),
                    mode='lines+markers',
                    name='NDVI Time Serie',
                    line=dict(color='#4E8C39', width=4)
                    ))
# Edit the layout
fig_ndvi.update_layout(title='Serie de tiempo NDVI',
                       xaxis_title='Mes',
                       yaxis_title='Índice de Vegetación')

fig_ndvi.show()